In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import pandas as pd

<h1># 01 - Ergebnisdaten: Parsen der Ergebnisdaten - Tokyo (2008 - 2014)</h1>
<hr>
<p><b>Hinweis:</b> Dateifpade sind absolut angegeben und müssen entsprechend der eigenen Verzeichnisstruktur angepasst werden!</p>
<hr>

In [ ]:
# Startnummern der 10 besten Athleten:innen ermitteln.
# Über die Startnummern werden die einzelnen Athleten:innen gesucht.
def get_biblist(year, ges):
    # Treiber laden
    driver_biblist = webdriver.Firefox()
    # Seite laden
    url = "https://www.marathon.tokyo/2014/index.php"
    driver_biblist.get(url)
    time.sleep(5)
    if ges == 'M':
        ges_nr = "1"
    elif ges == 'W':
        ges_nr = "2"
    else:
        ges_ner = "0"
         
    driver_biblist.find_element_by_xpath("//select[@id='holdingyear']/option[@value='" + str(year) + "']").click()
    time.sleep(3)
    driver_biblist.find_element_by_xpath("//div[@class='result']/p[" + ges_nr + "]/a[1]").click()
    time.sleep(5)
    xpath_row_main = "//div[@id='contents']/div[@class='result']/table[@class='rank']/tbody[1]"
    bib_list = []
    for r in range(2,12):
        bib_list.append(driver_biblist.find_element_by_xpath(xpath_row_main + "/tr[" + str(r) + "]/td[2]").text)
    
    print(bib_list)

    # Treiber schliessen
    driver_biblist.close()
    
    return bib_list

In [ ]:
for year in range(2008, 2015):
    # Treiber laden
    driver = webdriver.Firefox()
    # Seite laden
    url = "https://www.marathon.tokyo/2014/index.php"
    driver.get(url)
    time.sleep(5)

    # Leeres Dataframe anlegen
    df_results = pd.DataFrame(pd.DataFrame(
    columns=[
        'Jahr'
        ,'Ges'
        ,'Name'        
        ,'Number'
        ,'Time'
        ,'Time_Chip '
        ,'Place'
        ,'5K_Time'
        ,'10K_Time'
        ,'15K_Time'
        ,'20K_Time'
        ,'HK_Time'
        ,'25K_Time'
        ,'30K_Time'
        ,'35K_Time'
        ,'40K_Time'
        ,'FK_Time'
        ])
    ,dtype=str)
    
    for ges in ["M", "W"]:

        # Liste mit Startnummern der zehn besten Athleten:innen ermittelnt
        athlet_list = get_biblist(year, ges)
        
        # Uber die ersten 10 Athleten iterieren (Header uebersprungen)
        for a in athlet_list:
            driver.find_element_by_xpath("//select[@id='holdingyear']/option[@value='" + str(year) + "']").click()
            time.sleep(3)
            
            # Ein Athlet aufrufen        
            driver.find_element_by_xpath("//input[@name='number']").send_keys(a)
            time.sleep(2)
            driver.find_element_by_xpath("//input[@id='search_number']").click()
            time.sleep(4)
            
            xpath_athlet = "//div[@id='contents']/div[@class='result']"
            xpath_athlet_main = xpath_athlet + "/table[1]/tbody[1]"
            xpath_athlet_splits = xpath_athlet + "/table[2]/tbody[1]"
            
            
            # Auf der naechsten Seite die Daten abziehen
            
            result_temp = {
                'Jahr' : year
                ,'Ges' : ges
                ,'Name' : driver.find_element_by_xpath(xpath_athlet_main + "/tr[1]/td[1]").text        
                ,'Number' : driver.find_element_by_xpath(xpath_athlet_main + "/tr[2]/td[1]").text
                ,'Time' : driver.find_element_by_xpath(xpath_athlet_main + "/tr[3]/td[1]").text
                ,'Time_Chip ' : driver.find_element_by_xpath(xpath_athlet_main + "/tr[4]/td[1]").text
                ,'Place' : driver.find_element_by_xpath(xpath_athlet_main + "/tr[6]/td[1]").text
                ,'5K_Time' : driver.find_element_by_xpath(xpath_athlet_splits + "/tr[2]/td[3]").text
                ,'10K_Time' : driver.find_element_by_xpath(xpath_athlet_splits + "/tr[4]/td[3]").text
                ,'15K_Time' : driver.find_element_by_xpath(xpath_athlet_splits + "/tr[6]/td[3]").text
                ,'20K_Time' : driver.find_element_by_xpath(xpath_athlet_splits + "/tr[8]/td[3]").text
                ,'HK_Time' : driver.find_element_by_xpath(xpath_athlet_splits + "/tr[10]/td[3]").text
                ,'25K_Time' : driver.find_element_by_xpath(xpath_athlet_splits + "/tr[11]/td[3]").text
                ,'30K_Time' : driver.find_element_by_xpath(xpath_athlet_splits + "/tr[13]/td[3]").text
                ,'35K_Time' : driver.find_element_by_xpath(xpath_athlet_splits + "/tr[15]/td[3]").text
                ,'40K_Time' : driver.find_element_by_xpath(xpath_athlet_splits + "/tr[17]/td[3]").text
                ,'FK_Time' : driver.find_element_by_xpath(xpath_athlet_splits + "/tr[19]/td[3]").text
            }
            df_results = df_results.append(result_temp, ignore_index=True)

            # Zurueck zu der Athletenliste
            driver.find_element_by_xpath(xpath_athlet + "/p[@class='result-footer']/a[1]").click()
            time.sleep(3)

    
    # Ergebnisse anzeigen
    # print(df_results.head())

    # Daten in die CSV Exportieren
    file_path = '/home/paul/python_projects/masterthesis/data/Tokyo/'
    file_name = 'daten_wmm_tokyo_' + str(year) + '.csv'
    df_results.to_csv(file_path+file_name, sep=';', index=False)

    # Treiber schliessen
    driver.close()